# Introduction

The main theme is the modeling of glacier dynamics. It explores how changes in snowfall and surface altitude affect glacier stability. Kinematic wave theory is used to study how glaciers respond to sudden changes in accumulation, focusing on their sensitivity to climate changes.
It is important to understand glacier behavior as it helps predict ice loss, sea-level rise, and climate change impacts. This instability is important for modeling how glaciers respond to climate change.

STILL NEED: What observations, models, etc are used by others that are different from what you are doing in the project.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags

L = 1000 
c = 10
dx = 1 
dt = 1/10   
T = 100   
nx = int(L / dx)  
nt = int(T / dt)  
Cr = c * dt / dx
x = np.linspace(0, L, nx)
q_initial = np.sin( np.pi * x )
def accumulation(x, t):
    return 1/2-0.01*x